In [2]:
import h5py
import matplotlib
from matplotlib import pyplot as plt
from berg import BERG
import nibabel as nib
import numpy as np
import os
from PIL import Image
import torchvision
from torchvision import transforms as trn
from tqdm import tqdm
from IPython.display import display, JSON

berg_dir = "/Volumes/Extreme SSD/Datasets/neural-encoding-simulation-toolkit" 
test_images = "tutorial_images"

# 1 | Load and Prepare the Images for Generating the In Silico Neural Responses

With the following code you will load and preprocess the images that you will later use to generate the in silico neural responses using an encoding model of your choice.

These encoding model expects as input images in a specific format: a 4D numpy array with shape (Batch size × 3 RGB Channels × Width × Height) and integer values in the range [0, 255]. Because the images must also be of square size (equal width and height), you will center crop them during loading.

In [3]:
images_dir = test_images
images_list = os.listdir(images_dir)
images_list.sort()

images = []
for img in tqdm(images_list):
    img_dir = os.path.join(images_dir, img)
    img = Image.open(img_dir).convert('RGB')
    # Center crop the images to square format, and resize them
    transform = trn.Compose([
        trn.CenterCrop(min(img.size)),
        trn.Resize((227,227))
    ])
    img = transform(img)
    img = np.asarray(img)
    img = img.transpose(2,0,1)
    images.append(img)
images = np.asarray(images)

# Print the images dimensions
print('\n\nImages shape:')
print(images.shape)
print('(Batch size × 3 RGB Channels x Width x Height)')

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:00<00:00, 324.52it/s]



Images shape:
(100, 3, 227, 227)
(Batch size × 3 RGB Channels x Width x Height)


## 2.1 | Create the BERG object

To use the BERG package, you first need to create a `BERG` object, by providing the path to the Neural Encoding Simulation Toolkit directory. This object will be the instance through which you can generate in silico neural responses.

In [4]:
# Initialize the BERG object with the path to the toolkit directory
berg = BERG(berg_dir)

## 2.2 | Browse the available encoding models

The `list_models()` method lists the encoding models that are available in BERG. BERG contains several encoding models, defined by the following model ID naming convention:

`{modality}-{dataset}-{model}`

where

* **`modality`:** The neural recording recording modality on which the encoding model was trained.
* **`dataset`:** The neural dataset on which the encoding model was trained.
* **`model`:** The type of encoding model used.




In [5]:
# List all available models and their versions
available_models = berg.list_models()
print(f"Available models: {available_models}")

Available models: ['fmri-nsd-fwrf', 'fmri-nsd_fsaverage-vit_b_32', 'eeg-things_eeg_2-vit_b_32']


You can also view the modalities and datasets in a more structured format:

In [6]:
# Get a hierarchical view of available models by modality and dataset
catalog = berg.get_model_catalog(print_format=True)
print(f"Model Catalog as Dict: {catalog}")

Available Modalities and Datasets:
• EEG
  └─ THINGS EEG2

• FMRI
  └─ Natural Scenes Dataset (NSD) (fsaverage surface space)
  └─ Natural Scenes Dataset (NSD) (subject-native volume space)

Model Catalog as Dict: {'fMRI': ['Natural Scenes Dataset (NSD) (fsaverage surface space)', 'Natural Scenes Dataset (NSD) (subject-native volume space)'], 'EEG': ['THINGS EEG2']}


The `print_format=True` parameter displays a nicely formatted hierarchical view, making it easy to browse the available encoding models.

<font color='red'><b>NOTE:</b></font> For a list of all available encoding model, please see the [documentation](https://neural-encoding-simulation-toolkit.readthedocs.io/en/latest/models/overview.html).

In [7]:
# Select one of the existing models
model_id = 'eeg-things_eeg_2-vit_b_32'  #@param ["fmri-nsd-fwrf", "eeg-things_eeg_2-vit_b_32"]

## 2.3 | Get detailed model information

With the `describe()` you can get detailed information about this model and how to use it, including:

* **Basic Information:** Details about the modality, dataset, and encoding model.
* **Description:** How the model works and what it does.
* **Input Requirements:** Format specifications for input stimuli (dimensions, type, etc.).
* **Output Format:** The structure and meaning of the model's in silico neural predictions.
* **Parameters:** Required and optional arguments for the model's functions.
* **Performance Information:** Directory where to find the encodind model's prediction accuracy plots.
* **Usage Examples:** Code snippets showing how to use the model.

This information will help you understand how to properly set up and use the model in the following sections.

<font color='red'><b>NOTE:</b></font> You can also view this information on the model in the [documentation](https://neural-encoding-simulation-toolkit.readthedocs.io/en/latest/models/overview.html).

In [8]:
# Get comprehensive information about the fMRI model
model_info = berg.describe(model_id)
print(list(model_info.keys()))

🧠 Model: eeg-things_eeg_2-vit_b_32

Modality: EEG
Training dataset: THINGS EEG2
Creator: Alessandro Gifford

📋 Description:
These encoding models consist in a linear mapping (through linear regression) of
vision transformer (Dosovitskiy et al., 2020) image features onto EEG responses.
Prior to mapping onto EEG responses, the image features have been downsampled to
250 principal components using principal component analysis.  The encoding
models were trained on THINGS EEG2 (Gifford et al., 2022), 63-channel EEG
responses of 10 subjects to over 16,740 images from the THINGS initiative
(Hebart et al., 2019).  **Preprocessing**. During preprocessing the 63-channel
raw EEG data was filtered between 0.03 Hz and 100 Hz; epoched from -100 ms to
+600 ms with respect to stimulus onset; transformed using current source density
transform; downsampled to 200 Hz resulting in 140 times points per epoch (one
every 5 ms); baseline corrected at each channel using the mean of the pre-
stimulus interval. 

You can examine the parameters required for your specific model:

In [9]:
display(JSON(model_info["parameters"]))

<IPython.core.display.JSON object>

---
---


# 3 | The BERG Workflow: Generate In Silico Neural Responses

BERG has a consistent workflow across different neural modalities and models. This workflow comes down to just **two key functions**:

- `get_encoding_model()` – Load an encoding model of your choice.
- `encode()` – Generate in silico neural responses using the encoding model previously loaded.

<font color='red'><b>NOTE:</b></font> For a detailed description of these functions' input parameters and outputs, please see the [documentation](https://neural-encoding-simulation-toolkit.readthedocs.io/en/latest/models/overview.html).

## 3.1 | Load the encoding models with get_encoding_model()

You will first load your chosen model using `get_encoding_model()`:

```python
# General format
model = berg.get_encoding_model(model_id, subject, **other_parameters)
```

### Key parameters:
- `model_id`: The identifier for the model (e.g., "fmri-nsd-fwrf").
- `subject`: Subject ID ib which the encoding model was trained (required for all models).
- `device`: Computing device ("cpu", "cuda", or "auto").
- `selection`: Optional dictionary to specify the neural features for which the in silico neural responses are generated (e.g., specific fMRI ROIs or voxels, or specific EEG channels or time points).

In [11]:
# Load the selected model for subject 1
model = berg.get_encoding_model(model_id,
                               subject=1,
                               device="auto")

# # If you are using the 'fmri-nsd-fwrf' encoding model, you should select the
# # for which you want to generate the in silico fMRI responses.
# model = berg.get_encoding_model(model_id,
#                                 subject=1,
#                                 selection={"roi": "V1"}, # or other
#                                 device="cpu")

# # If you are using the 'fmri-nsd-fwrf' encoding model, you can select the
# # EEG channels and/or time points for which you want to generate the in silico
# # EEG responses.
# timepoint_mask = [1]*48 + [0]*92
# model = berg.get_encoding_model(model_id,
#                                 subject=1,
#                                 selection={
#                                     "channels": ["F7"],
#                                     "timepoints": timepoint_mask}
#                                 )

Model loaded on cpu for subject 1


## 3.2 | Generate the in silico neural responses with encode()

Once the model is loaded, you can use it to generate in silico neural responses to images using the `encode()` function:

```python
# Basic usage
responses = berg.encode(model, images)

# To also get metadata relative to the in silico neural responses
responses, metadata = berg.encode(model, images, return_metadata=True)
```

In [12]:
# Generate in silico neural responses for images
in_silico_responses = berg.encode(model, images)

Encoding EEG responses: 100%|██████████| 1/1 [00:01<00:00,  1.52s/it, Encoded images=100, Total images=100]


In [13]:
print(f"Response shape: {in_silico_responses.shape}")

Response shape: (100, 4, 63, 140)


## 3.3 | Load the metadata

BERG provides metadata for each model, which includes information on the neural data used to train the encoding models, and on the trained models themselves.

You can access this metadata in two ways:

In [14]:
# Method 1: Get metadata during encoding
in_silico_responses, metadata = berg.encode(model, images, return_metadata=True)

# Method 2: Get metadata directly from the encoding model with selection parameters
example_model = "fmri-nsd-fwrf"
example_subject = 1
example_roi = "V1"
metadata = berg.get_model_metadata(example_model, subject=example_subject, roi=example_roi)

Encoding EEG responses: 100%|██████████| 1/1 [00:01<00:00,  1.52s/it, Encoded images=100, Total images=100]


---
---
# 4 | Complete Working Examples

## Example 1: fMRI ROI selection

In this example you will use the `fmri-nsd-fwrf` encoding model.

In [15]:
#@title Select the fMRI encoding model's subject and ROI
subject = 1  #@param {type:"slider", min:1, max:8, step:1}
roi = "V1"  #@param ["V1", "V2", "V3", "hV4", "EBA", "FBA-2", "OFA", "FFA-1", "FFA-2", "PPA", "RSC", "OPA", "OWFA", "VWFA-1", "VWFA-2", "mfs-words", "early", "midventral", "midlateral", "midparietal", "parietal", "lateral", "ventral"]


In [16]:
# Initialize BERG
from berg import BERG
berg = BERG(berg_dir)

# Load the encoding model of the selected ROI and subject
fmri_model = berg.get_encoding_model("fmri-nsd-fwrf",
                                subject=subject,
                                selection={"roi": roi})

# Generate the in silico fMRI responses
insilico_fmri = berg.encode(fmri_model, images)
print(f"{roi} responses shape: {insilico_fmri.shape}")

# Print the in silico fMRI responses shape
import sys; sys.stdout.write("\n")
print(f"fMRI response shape: {insilico_fmri.shape}")

# Show metadata
metadata = berg.get_model_metadata("fmri-nsd-fwrf", subject=subject, roi=roi)
print(metadata.keys())

Model loaded on cpu for subject 1, ROI V1


Encoding fMRI responses: 100%|██████████| 1/1 [00:00<00:00,  1.02it/s, Encoded images=100, Total images=100]

V1 responses shape: (100, 1350)

fMRI response shape: (100, 1350)
dict_keys(['fmri', 'encoding_models'])


## Example 2: EEG channel and time point selection

In this example you will use the `eeg-things_eeg_2-vit_b_32` encoding model.

In [17]:
#@title Select EEG enconding model's subject, channels and time points

#@markdown Select Subject
subject = 1  #@param {type:"slider", min:1, max:10, step:1}

#@markdown Select the used EEG channels
# ['Fp1', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9',
# 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8',
# 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2',
# 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3',
# 'FCz', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz',
# 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4',
# 'FT8', 'F6', 'F2', 'AF4', 'AF8']

channels = ['F7', 'F3', 'F4', 'F8']  #@param {type:"raw"}

#@markdown Select the EEG time point range (between 0 and 140 timepoints)
start_timepoint = 0  #@param {type:"slider", min:0, max:139, step:1}
end_timepoint = 140  #@param {type:"slider", min:1, max:140, step:1}

# Create timepoint mask from start and end selection
timepoint_mask = [1 if start_timepoint <= i < end_timepoint else 0 for i in range(140)]


In [18]:
from berg import BERG

# Initialize BERG
berg = BERG(berg_dir)

# Load the EEG encoiding model
eeg_model = berg.get_encoding_model("eeg-things_eeg_2-vit_b_32",
                                    subject=subject,
                                    selection={
                                        "channels": channels,
                                        "timepoints": timepoint_mask
                                    })

# Generate the in silico EEG responses
insilico_eeg = berg.encode(eeg_model, images)

# Print the in silico EEG responses shape
import sys; sys.stdout.write("\n")
print(f"EEG response shape: {insilico_eeg.shape}")

# Show metadata
metadata = berg.get_model_metadata("eeg-things_eeg_2-vit_b_32", subject=subject)
print(metadata.keys())

Model loaded on cpu for subject 1


Encoding EEG responses: 100%|██████████| 1/1 [00:01<00:00,  1.45s/it, Encoded images=100, Total images=100]


EEG response shape: (100, 4, 4, 140)
dict_keys(['eeg', 'encoding_models'])
